# KOBART 모델 학습
Github에 작성되어 있는 KoBART 사전 학습 모델을 불러오고, AI-HUB에서 직접 수집한 데이터를 사용하여 Fine-tuning 시켰다.
원래 전처리한 데이터까지 포함하여 약 48만개의 데이터를 학습에 사용하려고 했지만, 시간과 GPU 환경의 한계로 학습 데이터는 2만개, 테스트 데이터는 4000개만 사용했다.
사용한 모델은 bin파일 형태로 저장하고, 이 파일을 불러와서 성능 평가나 문서 요약에 사용할 수 있다.

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [7]:
DATA_PATH = '....'

In [ ]:
cd /content/drive/.....

/content/drive/MyDrive/AI_camp/CP1


In [ ]:
# KoBART Pre trained 모델 로드
# !git clone https://github.com/seujung/KoBART-summarization.git

Cloning into 'KoBART-summarization'...
remote: Enumerating objects: 142, done.
remote: Counting objects: 100% (59/59), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 142 (delta 46), reused 37 (delta 37), pack-reused 83
Receiving objects: 100% (142/142), 37.23 MiB | 24.17 MiB/s, done.
Resolving deltas: 100% (76/76), done.


In [ ]:
!pip install git+https://github.com/SKT-AI/KoBART#egg=kobart

In [1]:
cd /content/drive/......

/content/drive/MyDrive/AI_camp/CP1/KoBART-summarization


In [ ]:
#필요한 패키지 설치
#cuda와 KoBART 버전에 맞게

!pip install torch==1.10.0
!pip install transformers==4.16.2
!pip install pytorch-lightning==1.5.10
!pip install streamlit==1.2.0

#downgrade 필수
!pip install torchmetrics==0.6.0

#이것도 매칭
!pip install torchvision==0.11.1

#torch 1.10.0이랑 매칭시켜야함
!pip install torchtext==0.6.0

#모델 불러오기 위한 설치
!pip install pyyaml==5.4.1

In [2]:
import torch
print("Torch version:{}".format(torch.__version__))
print("cuda version: {}".format(torch.version.cuda))
print("cudnn version:{}".format(torch.backends.cudnn.version()))

Torch version:1.10.0+cu102
cuda version: 10.2
cudnn version:7605


In [8]:
# Fine-tuning에 사용할 데이터
import pandas as pd

t = pd.read_csv(f'{DATA_PATH}train2.tsv', sep= '\t')
v = pd.read_csv(f'{DATA_PATH}test2.tsv', sep= '\t')

display(t.head())
display(v.head())

tcmalloc: large alloc 1073741824 bytes == 0x4634c000 @  0x7f99e5d8b2a4 0x7f99babae9a5 0x7f99babafcc1 0x7f99babb169e 0x7f99bab8250c 0x7f99bab8f399 0x7f99bab7797a 0x59afff 0x515655 0x549e0e 0x593fce 0x511e2c 0x549576 0x593fce 0x511e2c 0x593dd7 0x5118f8 0x549576 0x4bca8a 0x5134a6 0x549e0e 0x593fce 0x548ae9 0x5127f1 0x549576 0x604173 0x62a809 0x59358d 0x515244 0x598ef4 0x515a6e


,news,summary
0,ha당 조사료 400만원…작물별 차등 지원이성훈 sinawi@hanmail.net전...,전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 벼를 심었던 논에 벼 대...
1,"8억 투입, 고소천사벽화·자산마을에 색채 입혀이성훈 sinawi@hanmail.ne...",여수시는 컬러빌리지 사업에 8억원을 투입하여 ‘색채와 빛’ 도시를 완성하여 고소천사...
2,전남드래곤즈 해맞이 다짐…선수 영입 활발이성훈 sinawi@hanmail.net전남...,전남드래곤즈 임직원과 선수단이 4일 구봉산 정상에 올라 일출을 보며 2018년 구단...
3,"11~24일, 매실·감·참다래 등 지역특화작목이성훈 sinawi@hanmail.ne...","광양시는 농업인들의 경쟁력을 높이고, 소득안정을 위해 매실·감·참다래 등 지역특화작..."
4,"홍콩 크루즈선사‘아쿠아리우스’ 4, 6월 여수항 입항이성훈 sinawi@hanmai...",올해 4월과 6월 두 차례에 걸쳐 타이완의 크루즈 관광객 4000여명이 여수에 입항...


,news,summary
0,"[ 박재원 기자 ] '대한민국 5G 홍보대사'를 자처한 문재인 대통령은 ""넓고, 체...",8일 서울에서 열린 5G플러스 전략발표에 참석한 문재인 대통령은 5G는 대한민국 혁...
1,] 당 지도부 퇴진을 놓고 바른미래당 내홍이 격화되고 있다.바른미래당이 8일 연 최...,8일 바른미래당 최고의원 회의에 하태경 의원 등 5명의 최고의원이 지도부 퇴진을 요...
2,[ 홍윤정 기자 ] 8일 서울 올림픽공원 K아트홀.지난 3일 한국이 세계 최초로 5...,지난 3일 한국이 세계 첫 5세대 이동통신 서비스를 보편화한 것을 축하하는 '코리안...
3,] 박원순 서울시장(사진)이 8일 고층 재개발·재건축 관련 요구에 작심한 듯 쓴소리...,박원순 서울시장은 8일 서울시청에서 열린 '골목길 재생 시민 정책 대화'에 참석하여...
4,"[ 임근호 기자 ] ""SK(주)와 미국 알파벳(구글 지주회사)의 간결한 지배구조를 ...",주주가치 포커스를 운용하는 KB자산운용이 SK와 알파벳(구글 지주회사)의 모범적 ...


In [27]:
# 모든 데이터를 다 사용하기에는 학습이 오래 걸려서
# 학습에는 20000, 테스트에는 4000개의 데이터만 사용
t_real = t.sample(n=20000, random_state = 42)
v_real = v.sample(n=4000, random_state = 42)

display(t_real.head())
display(v_real.head())

,news,summary
375828,황신섭 구리시가 정부의 상권활성화 공모 사업에 사활을 걸고 있다.지원금 80억원이 ...,구리시가 정부의 지원금 80억원이 걸린 상권활성화 공모 사업에 선정되기 위해 사활을...
255749,"ㆍ신규 공급 물량을 둘러싼 정반대 시각… 어느 쪽 주장이 맞나""하물며 달팽이도 집이...","세 차례 발표한 정부 대책에도 서울 집값은 고공행진, 이와 같은 현상에 대한 원인 ..."
184664,홍성용 기자◆ 조국 임명 강행 ◆이해찬 더불어민주당 대표(가운데)가 9일 국회에서 ...,민주당은 조국 장관이 임명된 9일 검찰을 향한 공세 수위를 높였는데 이해찬 당대표의...
91811,"이상우 신동근·윤상현, 의료법 개정안고 임세원 교수의 사망사고를 계기로 의료기관 내...",인천지역 여야 의원들은 임세원 교수의 사망사고를 계기로 의료인과 환자의 보호를 위한...
161485,LH(한국토지주택공사)가 이달 종합심사낙찰제(이하 종심제) 건축공사를 대거 발주한다...,LH(한국토지주택공사)가 이달 종합심사낙찰제(이하 종심제) 건축공사를 대거 발주를 ...


,news,summary
10015,아침 저녁으로 쌀쌀한 날씨가 이어지면서 편의점 업계가 동절기 모드 돌입에 나선 가운...,세븐일레븐은 동절기를 맞아 1만개 한정 판매할 예정인 경량패딩조끼를 업계 최초로 선...
45493,"(주)알파돔시티는 판교역 지상부에 판교만의 상징성을 부여하고, 활성화된 공간 조성을...",(주)알파돔시티는 오는 2021년까지 판교역 지상부에 판교만의 상징성을 나타내는 쾌...
27397,미투(#Me Tooㆍ나도 피해자다) 운동이 일상화되면서 미국 브로드웨이에 '인티머시...,미투 운동 일상화로 브로드웨이에 '인티머시 디렉터'라는 직업이 등장했는데 이는 '성...
5256,부산글로벌테크비즈센터(B-TBC) 개관식에 참석한 오거돈 부산시장과 유영민 과학기술...,과학기술정보통신부와 부산시 및 연구개발특구진흥재단이 31일 부산 미음일반산단 R&D...
24393,삼성전기가 초슬림 광학 5배줌 카메라 모듈 개발에 성공해 이달부터 대량생산에 들어간...,20일 삼성전기가 초슬림 광학 5배줌 카메라 모듈 개발에 성공했다고 밝혔는데 이번 ...


In [28]:
t_real.to_csv(f'{DATA_PATH}train.tsv', header = True, sep = '\t', index = False)
v_real.to_csv(f'{DATA_PATH}test.tsv', header = True, sep = '\t', index = False)

In [29]:
#Fine-tuning
!python train.py --gradient_clip_val 1.0 --max_epochs 5 --default_root_dir logs --gpus 1 --batch_size 8 --num_workers 4 --lr 10e-5 --warmup_ratio 0.2

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
INFO:root:Namespace(accelerator=None, accumulate_grad_batches=None, amp_backend='native', amp_level=None, auto_lr_find=False, auto_scale_batch_size=False, auto_select_gpus=False, batch_size=8, benchmark=False, check_val_every_n_epoch=1, checkpoint_callback=None, checkpoint_path=None, default_root_dir='logs', detect_anomaly=False, deterministic=False, devices=None, enable_checkpointing=True, enable_model_summary=True, enable_progress_bar=True, fast_dev_run=False, flush_logs_every_n_steps=None, gpus=1, gradient_clip_algorithm=None, gradient_clip_val=1.0, ipus=None, limit_predict_batches=1.0, limit_test_batches=1.0, limit_train_batches=1.0, limit_val_batches=1.0, log_eve

In [30]:
# kobart_summary라는 폴더 안에 Fine tuning 됨 모델을 불러옴
!python get_model_binary.py --hparams ./logs/tb_logs/default/version_0/hparams.yaml --model_binary ./logs/model_chp/epoch=01-val_loss=1.628.ckpt

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
get_model_binary.py:13: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  hparams = yaml.load(f)


In [31]:
# Fine tuning한 모델로 성능 테스트 해보기
import torch
from transformers import PreTrainedTokenizerFast
from transformers.models.bart import BartForConditionalGeneration

device = torch.device("cuda")
model = BartForConditionalGeneration.from_pretrained('./kobart_summary')
model.load_state_dict(torch.load('/content/drive/MyDrive/AI_camp/CP1/KoBART-summarization/kobart_summary/pytorch_model.bin'))
model.to(device)

tokenizer = PreTrainedTokenizerFast.from_pretrained('gogamza/kobart-base-v1')

In [36]:
%%time
text = """
SK텔레콤은 우리말 인공지능(AI) 언어모델의 개발 역량 향상과 국어 정보화 저변 확대를 위해 ‘인공지능 언어 능력 평가 대회’를 개최한다고 1일 밝혔다. 
이번 대회는 SK텔레콤·문화체육관광부·국립국어원 함께 주최한다. 대회는 9월 1일 오후 SK텔레콤의 기본 AI 언어모델과 국립국어원의 평가 데이터 세트 공개와 함께 시작되며, 
참가자들은 오는 9월 15일부터 11월 1일까지 결과물을 수시로 업데이트하여 제출할 수 있다. 이번 대회는 개인 또는 팀으로 누구나 참가할 수 있다. 참가팀 중 대상(문화체육관광부 장관상), 
금상·은상·동상, 특별상을 선정할 계획이며, 수상자 전원에게는 매년 SK텔레콤이 개최하는 AI 펠로우십(Fellowship)의 서류 심사 면제 특권도 제공할 계획이다. 참가자들은 제공된 언어 모델 등을 기반으로 
각각의 AI 언어모델 프로그램을 개발하여 우리말을 이해하고 분석하는 능력을 평가받는다. 제출된 언어모델을 평가하는 과제는 4가지다. ▲문장의 문법 오류 판단하기(문장 적법성 판단) ▲맥락별 단어 의미 구별하기(동형이의어 구별) 
▲문장 읽고 원인 추론하기(인과 관계 추론) ▲제시문 읽고 질문에 예/아니오 답하기(판정 의문문) 등이다. 이번 평가는 지금까지 한국어 인공지능 모델 평가를 위해 공개된 데이터세트들보다 난이도가 다소 높은 내용으로 구성됐다. 
SK텔레콤에서 제공하는 언어모델은 매개 변수가 12억개인 모델이다. 지난해 공개한 KoGPT2 모델보다 약 8배 크다. 이는 SK텔레콤이 국립국어원과 진행하고 있는 한국어 범용언어모델(GLM) 연구 과제의 초기 산출물로, 
기존에 SK텔레콤이 개발해 발표한 KoBERT, KoGPT2, KoBART 모델에 이어 한국어 AI 모델을 개발·활용하고자 하는 이들에게 도움이 될 것으로 기대된다. 에릭 데이비스 SK텔레콤 Language Superintelligence Labs장은 
“SK텔레콤과 국립국어원이 협업하여 준비한 이번 경진대회가 언어와 AI에 대한 역량을 맘껏 펼치는 장이 되길 기대한다”며 “나아가 이러한 건전한 경쟁이 범용언어모델을 비롯한 한국어 언어모델의 발전과 국어 정보화 확산에 기여하길 바란다”고 말했다.
"""

l1 = [text] * 1
output_list = []
for i in l1:
    input_ids = tokenizer.encode(i)
    input_ids = torch.tensor(input_ids, device = torch.device('cuda'))
    input_ids = input_ids.unsqueeze(0)
    output = model.generate(input_ids, eos_token_id=1, max_length=64, num_beams=2)
    output = tokenizer.decode(output[0], skip_special_tokens=True)
    output_list.append(output)

CPU times: user 368 ms, sys: 256 µs, total: 368 ms
Wall time: 366 ms


In [37]:
output_list[0]

'SK텔레콤은 우리말 인공지능(AI) 언어모델의 개발 역량 향상과 국어 정보화 저변 확대를 위해 ‘인공지능 언어 능력 평가 대회’를 개최한다고 밝혔으며, 참가자들은 제공된 AI 언어 모델 등을 기반으로 각각의 AI 언어모델 프로그램을 개발하여 우리말을 이해하고 분석하는 능력을 평가받는다.'

In [40]:
label = ["""
SK텔레콤은 문화체육관광부·국립국어원과 함께 우리말 인공지능(AI) 언어모델의 개발 역량 향상과 국어 정보화 저변 확대를 위해 ‘인공지능 언어 능력 평가 대회’를 개최한다고 밝히며 참가자들은 
제공된 언어 모델 등을 기반으로 각각의 AI 언어모델 프로그램을 개발하여 우리말을 이해하고 분석하는 능력을 평가받는다.
"""]

In [38]:
!pip install rouge

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [41]:
from rouge import Rouge

rouge = Rouge()
rouge.get_scores(output_list,label , avg=True)

{'rouge-1': {'r': 0.8918918918918919,
  'p': 0.9705882352941176,
  'f': 0.9295774597976593},
 'rouge-2': {'r': 0.8108108108108109,
  'p': 0.8571428571428571,
  'f': 0.8333333283371915},
 'rouge-l': {'r': 0.8918918918918919,
  'p': 0.9705882352941176,
  'f': 0.9295774597976593}}